# Init 

In [8]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Slice Creation

In [19]:
# Slice metadata
slice_name = 'dpdk-slice'
site = 'MAX'
num_nodes = 2
network_name='net'

In [10]:
# Slice creation
try:    
    i = 0
    ifaces = []
    slice = fablib.new_slice(name=slice_name)
    while i < num_nodes:
        print(f'Creating Node{i}')
        node = slice.add_node(name=f'Node_{i}', site=site, image='default_ubuntu_20')
        node.set_capacities(cores=8, ram=16, disk=20)
        ifaces.append(node.add_component(model='NIC_Basic', name=f'nic{i}').get_interfaces()[0])
        i += 1
    net = slice.add_l2network(name=network_name, interfaces=ifaces)
    print("Submitting slice request")
    slice.submit(wait_timeout=3000, wait_interval=30,)
except Exception as e:
    print(f"Exception: {e}")


-----------  ------------------------------------
Slice Name   dpdk-slice
Slice ID     98549972-4d27-41fc-81fa-34d0c554d951
Slice State  StableOK
Lease End    2022-09-28 01:51:32 +0000
-----------  ------------------------------------

Retry: 5, Time: 186 sec

ID                                    Name    Site    Host                         Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  -------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
350933dd-cb23-4465-a70f-89c7ccac5c3e  Node_0  MAX     max-w4.fabric-testbed.net        8     16     100  default_ubuntu_20  63.239.135.99    Active
eb8b730f-cd55-45ef-af34-270b47846d66  Node_1  MAX     max-w4.fabric-testbed.net        8     16     100  default_ubuntu_20  63.239.135.69    Active

Time to stable 186 seconds
Running post_boot_config ... Time to post boot config 197 seconds

Name            Node    Network  

# NIC Drivers

In [11]:
try:
    for node in slice.get_nodes():
        print(f'Installing network drivers on node {node.get_name()}')
        stdout, stderr = node.execute('export DEBIAN_FRONTEND=noninteractive')
        stdout, stderr = node.execute('sudo wget -nc -q -P /usr/src https://content.mellanox.com/ofed/MLNX_OFED-5.7-1.0.2.0/MLNX_OFED_LINUX-5.7-1.0.2.0-ubuntu20.04-x86_64.tgz')
        stdout, stderr = node.execute('sudo tar -xvf /usr/src/MLNX_OFED_LINUX-5.7-1.0.2.0-ubuntu20.04-x86_64.tgz')
        stdout, stderr = node.execute('sudo rm -rf /etc/apt/sources.list.d/mlnx_ofed.list && sudo touch /etc/apt/sources.list.d/mlnx_ofed.list')
        stdout, stderr = node.execute("echo 'deb file:/home/ubuntu/MLNX_OFED_LINUX-5.7-1.0.2.0-ubuntu20.04-x86_64/DEBS ./' | sudo tee -a /etc/apt/sources.list.d/mlnx_ofed.list")
        stdout, stderr = node.execute('sudo wget -qO - http://www.mellanox.com/downloads/ofed/RPM-GPG-KEY-Mellanox | sudo apt-key add -')
        stdout, stderr = node.execute('sudo apt-get update && sudo apt-get install -y -q mlnx-ofed-all mlnx-fw-updater')
        stdout, stderr = node.execute('sudo /etc/init.d/openibd restart')
        print(stderr)
except Exception as e:
    print(f"Exception: {e}")

Installing network drivers on node Node_0

Installing network drivers on node Node_1



# DPDK Installation

In [22]:
try:
    for node in slice.get_nodes():
        print(f'\nInstalling DPDK on node {node.get_name()}')
        stdout, stderr = node.execute('echo 4096 | sudo tee /sys/devices/system/node/node0/hugepages/hugepages-2048kB/nr_hugepages')
        stdout, stderr = node.execute('sudo mkdir -p /mnt/huge; sudo mount -t hugetlbfs pagesize=4GB /mnt/huge')
        stdout, stderr = node.execute('sudo apt-get install -y python3-pip linux-generic libelf-dev numactl libnl-3-dev libnuma-dev libpcap-dev libfdt-dev libbsd-dev libssl-dev zlib1g-dev')
        stdout, stderr = node.execute('sudo python3 -m pip install pyelftools meson ninja')
        stdout, stderr = node.execute('cd /usr/src/ && sudo git clone https://github.com/DPDK/dpdk && cd dpdk && sudo git checkout v20.11')
        stdout, stderr = node.execute('cd /usr/src/dpdk/ && sudo meson -Dexamples=all -D disable_drivers=net/mlx4 build && sudo ninja -j 20 -C build && sudo ninja -j 20 -C build install')
        stdout, stderr = node.execute('cd /usr/src/dpdk/build/examples/ && sudo ./dpdk-helloworld')
        print(stdout)
except Exception as e:
    print(f"Exception: {e}")


Installing DPDK on node Node_0
hello from core 1
hello from core 2
hello from core 3
hello from core 4
hello from core 5
hello from core 6
hello from core 7
hello from core 0


Installing DPDK on node Node_1
hello from core 1
hello from core 2
hello from core 3
hello from core 4
hello from core 5
hello from core 6
hello from core 7
hello from core 0



# Delete Slice

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.wait_ssh()
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")